**MINOR PROJECT**

Prediction in time series dataset

**SECTION 1: READING DATA**

In [20]:
import pandas as pd
import numpy as np
import math
df = pd.read_excel('./DATASET.xlsx')
df.dtypes


Unnamed: 0     object
year            int64
Para-1        float64
Para-2        float64
Para-3        float64
Para-4        float64
Para-5          int64
Para-6          int64
Para-7          int64
Para-8          int64
Para-9        float64
Para-10       float64
Para-11       float64
Para-12         int64
Para-13       float64
dtype: object

**SECTION 2: REMOVING IRREVELANT ENTRIES**

***1.) REMOVED SECTIONS WHERE DATA FOR 10TH ROW IS ABSENT***

***2.) CREATING SEPARATE SECTION FILES OF VALID SECTIONS***

In [21]:
import os
path = './sections/'
isExist = os.path.exists(path)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(path)

In [22]:
df2 = pd.DataFrame()
sec_no = ""
i = 0
new_df = pd.DataFrame()
while i < len(df):
    j = 0
    if type(df.iloc[i][0])==str:
        sec_no = df.iloc[i][0]
        j+=1
    j+=i
    exist = False
    while j < len(df) and type(df.iloc[j][0])!=str:
        if df.iloc[j][1] == 10:
            exist = True
        j += 1
    
    if(exist):
        df2 = df2.append(df.loc[i:j-1,:])
        new_df = new_df.append(df.loc[i:j-1,:])
        new_df.to_csv('./sections/'+sec_no+'.csv',index=False)
        new_df = new_df.iloc[0:0]
    i = j

df2.reset_index(inplace=True,drop=True)


**SECTION 3: ANALYSIS PHASE**

In [23]:
#importing libraries

from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [38]:
#df2.drop(columns=['Unnamed: 0'],inplace=True)
df2.dropna(axis = 0, how ='any', thresh = None, subset = None, inplace=True)

In [ ]:
df2.isna().sum()

In [62]:
import glob
path = "./sections/*.csv"
for fname in glob.glob(path):
    tdf = pd.read_csv(fname)
    tdf.drop(columns=['Unnamed: 0'],inplace=True)
    tdf.dropna(axis = 0, how ='any', thresh = None, subset = None, inplace=True)
    sec_no = fname.split('/')[2].split('.')[0]

    rmse_df = pd.DataFrame()


    for i in range(9,14):
        folder_path = './rmse comparisons/'
        isExist = os.path.exists(folder_path)
        if not isExist:
            os.makedirs(folder_path)

        x = tdf.drop(columns=['Para-9', 'Para-10',	'Para-11',	'Para-12',	'Para-13'], axis=1)
        y = tdf['Para-'+str(i)]
        x_train, x_test = x.iloc[:-1, :], x.iloc[-1:, :]
        y_train, y_test = y.iloc[:-1], y.iloc[-1:]

        model = XGBRegressor(booster='dart',objective='reg:squarederror',random_state=21,verbosity = 0)
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        rmse = np.round(math.sqrt(mean_squared_error(y_test, predictions)), 3)
        actual = y_test.values
        rmse_df.loc[0, 'Para-'+str(i)] = actual[0]
        rmse_df.loc[1, 'Para-'+str(i)] = np.round(predictions[0], 3)
        rmse_df.loc[2, 'Para-'+str(i)] = rmse
    
    rmse_df.to_csv(folder_path+'/'+sec_no+'.csv', index=False)

**SECTION 4: PLOTTING DATA and CALCUATING ERRORS**

In [63]:
rmse_collection = []

folder_path = './graphs/'
isExist = os.path.exists(folder_path)
if not isExist:
  os.makedirs(folder_path)

path = "./rmse comparisons/*.csv"
for fname in glob.glob(path):
  rmse_df = pd.read_csv(fname)
  sec_no = fname.split('/')[2].split('.')[0]

  mean_rmse = np.round(rmse_df.iloc[2, :].mean(axis=0), 3)
  rmse_collection.append(mean_rmse)
  line_1 = rmse_df.iloc[0, :]
  line_2 = rmse_df.iloc[1, :]
  fig, ax = plt.subplots()
  ax.plot(line_1, color = 'green', label = 'Actual Data')
  ax.plot(line_2, color = 'red', label = 'Predicted Data')
  plt.title("RMSE Graph: "+str(mean_rmse))
  ax.legend()
  plt.savefig("./graphs/"+str(sec_no)+".png")
  plt.close()

In [64]:
# FINAL RMSE ERROR

f_rmse = sum(rmse_collection)/len(rmse_collection)
print("RMSE: ", np.round(f_rmse, 3))

RMSE:  9.024
